In [1]:
from __future__ import print_function
import numpy as np
import h5py
import pickle as pkl
import random
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
from torch.distributions import normal
import itertools
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import h5py
import matplotlib as mpl
import pyfftw
import Pk_library as PKL

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [96]:
def read_halo_file(catalogue_path = "halo_catalogues/",
                   redshift = 0,
                  ):
    path = "{}Halo_catalogue_z={}.hdf5".format(catalogue_path, 
                                                redshift)
    
    f = h5py.File(path)
    
    mass = f["mass"]
    position_x = np.array(f["pos"])[:,0]
    position_y = np.array(f["pos"])[:,1]
    position_z = np.array(f["pos"])[:,2]
    radius = f["radius"]
    
    return np.array(mass), position_x, position_y, position_z, np.array(radius)

In [4]:
datapath=""
f = h5py.File("fields_z=0.0.hdf5", 'r')
f=f['delta_HI']

In [164]:
NUM_REDSHIFTS = 5
from collections import defaultdict

def build_redshift_data(halo_catalogue_path):
    
    redshift_dict = defaultdict(dict)
    
    for i in range(NUM_REDSHIFTS + 1):
        redshift_dict[i]["total_mass"], redshift_dict[i]["position_x"], redshift_dict[i]["position_y"], \
        redshift_dict[i]["position_z"], redshift_dict[i]["radius"] = \
            [*read_halo_file(halo_catalogue_path, i)] 
        
        redshift_dict[i]["HI_mass"] = np.array(pd.DataFrame(pd.read_csv("MHI_z={}.csv".format(i)))["0"])
        
    [df_0, df_1, df_2, df_3, df_4, df_5] = [pd.DataFrame({"total_mass":redshift_dict[i]["total_mass"],
                                                         "position_x":redshift_dict[i]["position_x"],
                                                         "position_y":redshift_dict[i]["position_y"],
                                                         "position_z":redshift_dict[i]["position_z"],
                                                         "radius":redshift_dict[i]["radius"],
                                                         "HI_mass":redshift_dict[i]["HI_mass"]}) \
                                           for i in range(NUM_REDSHIFTS+1)]
        
    return df_0, df_1, df_2, df_3, df_4, df_5

In [128]:
# df_0.head()

,HI_mass,position_x,position_y,position_z,radius,total_mass
0,1.321760e+11,0.856017,26.345181,18.318504,1.373196,4.108502e+14
1,1.256623e+11,19.696722,47.432613,48.919952,1.398435,3.463541e+14
2,1.204984e+11,23.862780,33.081966,41.393356,1.150424,3.010118e+14
3,1.176074e+11,10.904087,50.859200,47.598503,1.314615,2.777093e+14
4,1.166619e+11,68.559006,59.030533,53.987701,1.055330,2.703974e+14


In [165]:
redshift_dfs = build_redshift_data("halo_catalogues/")

for i in range(NUM_REDSHIFTS+1):
    print ("{}/{}".format(i, NUM_REDSHIFTS))
    redshift_dfs[i].to_csv("redshift_df_{}.csv".format(i), header=True, index=False)

0/5
1/5
2/5
3/5
4/5
5/5


In [131]:
(64/2048)*75 # mpc/h

2.34375

In [132]:
2048-64

1984

In [271]:
def get_halo_tensor(pos_x, pos_y, pos_z, redshift, sample_edge):
    
    """Args:
    - pos_x: left corner x coordinate in simulation, (int in (0, 2048-sample_edge))
    - pos_y: left corner y coordinate in simulation, (int in (0, 2048-sample_edge))
    - pos_z: left corner z coordinate in simulation, (int in (0, 2048-sample_edge))
    
    - redshift: redshift, (int in range(NUM_REDSHIFTS+1))
    - sample_edge: size of one edge of each sample (int: sample.size[0])"""
    
    new_halo_tensor = np.zeros((sample_edge,)*3)
    redshift_df = pd.DataFrame(pd.read_csv("redshift_df_{}.csv".format(redshift)))
    halo_mpc_edge = (sample_edge/2048)*75
    halo_mpc_voxel_edge = (1/2048)*75
    halo_pos_x, halo_pos_y, halo_pos_z = [(x/2048)*75 for x in [pos_x, pos_y, pos_z]]
    upper_x, upper_y, upper_z = [pos + halo_mpc_edge for pos in [halo_pos_x, halo_pos_y, halo_pos_z]]
    
    cube_in_position = redshift_df[(redshift_df["position_x"] >= halo_pos_x)&\
                                   (redshift_df["position_x"] < upper_x)&\
                                   (redshift_df["position_y"] >= halo_pos_y)&\
                                   (redshift_df["position_y"] < upper_y)&\
                                   (redshift_df["position_z"] >= halo_pos_z)&\
                                   (redshift_df["position_z"] < upper_z)]
    
    edge_r = [*range(sample_edge)]
    voxels = [*itertools.product(edge_r, edge_r, edge_r)]
    
    for voxel in voxels:
        x, y, z = voxel[0], voxel[1], voxel[2]
        new_halo_tensor[x, y, z] = np.sum(cube_in_position[(cube_in_position["position_x"]>=halo_mpc_voxel_edge*x)&\
                                                                   (cube_in_position["position_x"]<halo_mpc_voxel_edge*x + halo_mpc_voxel_edge)&\
                                                                   (cube_in_position["position_y"]>=halo_mpc_voxel_edge*y)&\
                                                                   (cube_in_position["position_y"]<halo_mpc_voxel_edge*y + halo_mpc_voxel_edge)&\
                                                                   (cube_in_position["position_z"]>=halo_mpc_voxel_edge*z)&\
                                                                   (cube_in_position["position_z"]<halo_mpc_voxel_edge*z + \
                                                                    halo_mpc_voxel_edge)]["HI_mass"])
                
    path = "large_halo_cubes/redshift_{}".format(redshift)
    with open("{}/lower_corner_{}_{}_{}.pkl".format(path, pos_x, pos_y, pos_z),"wb") as f:
        pkl.dump(new_halo_tensor, f)

In [273]:
edge_r = [*np.arange(0, 2048, 64)]

In [276]:
lower_corners = [*itertools.product(edge_r, edge_r, edge_r)]

In [277]:
split = int(len(lower_corners)/4)

In [278]:
split

8192

In [ ]:
# redshift_0
for corner in lower_corners[:split]:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 0, 64)

corner: (0, 0, 0), 0/32768


In [ ]:
# redshift_1
for corner in lower_corners:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 1, 64)

In [ ]:
# redshift_2
for corner in lower_corners:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 2, 64)

In [ ]:
# redshift_3
for corner in lower_corners:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 3, 64)

In [ ]:
# redshift_4
for corner in lower_corners:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 4, 64)

In [ ]:
# redshift_5
for corner in lower_corners:
    print ("corner: {}, {}/{}".format(corner, lower_corners.index(corner), len(lower_corners)))
    get_halo_tensor(corner[0], corner[1], corner[2], 5, 64)

### Power Spectrum Plots

In [20]:
def power_spectrum_hod(simulation_batch, generated_batch, halo_batch,
                       threads=1, MAS="CIC", axis=0, BoxSize=75.0/2048*64):
    
    """Args:
    - simulation_batch: (tensor) "real" sample batch taken from the data
    - halo_batch: (tensor) samples taken from the same locations of the halo catalogue of the
                  corresponding redshift
    - generated_batch: model output batch that corresponds to the simulation batch
    
       Returns:
    - power spectrum plot of simulation_batch, generated_batch, and halo batch
            mean +- 1 standard deviation for each 
       
       """
    
    # initialize batch lists
    k3d_real, k3d_gen, k3d_halo = [], [], []
    Pk_real, Pk_gen, Pk_halo = [], [], []
    BoxSize, axis, MAS, threads = BoxSize, axis, MAS, threads
    batch_size = simulation_batch.shape[0]

    for i in range(batch_size):

        simulation_sample, generated_sample, halo_sample = simulation_batch[i], generated_batch[i], halo_batch[i]
        delta_real_cube, delta_gen_cube, delta_halo = simulation_sample, generated_sample, halo_sample

        delta_real_cube /= np.mean(delta_real_cube, dtype=np.float64)
        delta_real_cube -= 1.0
        delta_real_cube = delta_real_cube.astype(np.float32)
        Pk_real_cube = PKL.Pk(delta_real_cube, BoxSize, axis, MAS, threads)

        delta_gen_cube /= np.mean(delta_gen_cube, dtype=np.float64)
        delta_gen_cube -= 1.0
        delta_gen_cube = delta_gen_cube.astype(np.float32)
        Pk_gen_cube = PKL.Pk(delta_gen_cube, BoxSize, axis, MAS, threads)

        delta_halo /= np.mean(delta_halo, dtype=np.float64)
        delta_halo -= 1.0
        delta_halo = delta_halo.astype(np.float32)
        Pk_halo = PKL.Pk(delta_halo, BoxSize, axis, MAS, threads)

        k3d_real.append(np.log(Pk_real_cube.k3D))
        k3d_gen.append(np.log(Pk_gen_cube.k3D))
        k3d_halo.append(np.log(Pk_halo.k3D))

        Pk_real.append(np.log(Pk_real_cube.Pk[:,0]))
        Pk_gen.append(np.log(Pk_gen_cube.Pk[:,0]))
        Pk_halo.append(np.log(Pk_halo.Pk[:,0]))

    [k3d_real, k3d_gen, k3d_halo] = [np.array(x) for x in [k3d_real, k3d_gen, k3d_halo]]
    [Pk_real, Pk_gen, Pk_halo] = [np.array(x) for x in [Pk_real, Pk_gen, Pk_halo]] 

    mean_k3d_real, mean_k3d_gen, mean_k3d_halo = np.mean(k3d_real, 0), np.mean(k3d_gen, 0), np.mean(k3d_halo, 0)
    stdev_k3d_real, stdev_k3d_gen, stdev_k3d_halo = np.std(k3d_real, 0), np.std(k3d_gen, 0), np.std(k3d_halo, 0)

    mean_Pk_real, mean_Pk_gen, mean_Pk_halo = np.mean(Pk_real, 0), np.mean(Pk_gen, 0), np.mean(Pk_halo, 0)
    stdev_Pk_real, stdev_Pk_gen, stdev_Pk_halo = np.std(Pk_real, 0), np.std(Pk_gen, 0), np.std(Pk_halo, 0)

    # Plotting
    plt.figure(figsize=(10,5))

    plt.rcParams["font.size"] = 12
    # real
    plt.plot(mean_k3d_real, mean_Pk_real, color="b", label="Original", linewidth=3, alpha=0.7, marker="o")
    plt.plot(mean_k3d_real, mean_Pk_real + stdev_Pk_real, color="b", linewidth=2.5, alpha=0.4, linestyle="--")
    plt.plot(mean_k3d_real, mean_Pk_real - stdev_Pk_real, color="b", linewidth=2.5, alpha=0.4, linestyle="--")

    # generated
    plt.plot(mean_k3d_gen, mean_Pk_gen, color="r", label="Generated", linewidth=3, alpha=0.7, marker="o")
    plt.plot(mean_k3d_gen, mean_Pk_gen + stdev_Pk_gen, color="r", linewidth=2.5, alpha=0.4, linestyle="--")
    plt.plot(mean_k3d_gen, mean_Pk_gen - stdev_Pk_gen, color="r", linewidth=2.5, alpha=0.4, linestyle="--")



    # hod
    plt.plot(mean_k3d_halo, mean_Pk_halo, color="k", label="HOD", linewidth=3, alpha=0.7, marker="o")
    plt.plot(mean_k3d_halo, mean_Pk_halo + stdev_Pk_halo, color="k", linewidth=3.5, alpha=0.4)
    plt.plot(mean_k3d_halo, mean_Pk_halo - stdev_Pk_halo, color="k", linewidth=3.5, alpha=0.4)
    
    plt.title("Power Spectrum Comparison")
    plt.xlabel('log(Pk.k3D)')
    plt.ylabel('log(Pk)')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=3)
    
    plt.show()


In [65]:
def power_spectrum_simulation(simulation_batch, threads=1, MAS="CIC", axis=0, BoxSize=75.0/2048*64):
    
    # initialize batch lists
    k3d_real, k3d_gen, k3d_halo = [], [], []
    Pk_real, Pk_gen, Pk_halo = [], [], []
    BoxSize, axis, MAS, threads = BoxSize, axis, MAS, threads
    batch_size = simulation_batch.shape[0]

    for i in range(batch_size):

        simulation_sample = np.array(simulation_batch[i])
        delta_real_cube = simulation_sample

        delta_real_cube /= np.mean(delta_real_cube, dtype=np.float64)
        delta_real_cube -= 1.0
        delta_real_cube = delta_real_cube.astype(np.float32)
        Pk_real_cube = PKL.Pk(delta_real_cube, BoxSize, axis, MAS, threads)
        k3d_real.append(np.log(Pk_real_cube.k3D))
        Pk_real.append(np.log(Pk_real_cube.Pk[:,0]))

    k3d_real, Pk_real = np.array(k3d_real), np.array(Pk_real)

    mean_k3d_real, stdev_k3d_real = np.mean(k3d_real, 0), np.std(k3d_real, 0)

    mean_Pk_real = np.mean(Pk_real, 0)
    stdev_Pk_real = np.std(Pk_real, 0)
    
    # Plotting
    plt.figure(figsize=(20,10))
    plt.rcParams["font.size"] = 14
    # real
    plt.plot(mean_k3d_real, mean_Pk_real, color="b", label="Original", linewidth=3, alpha=0.7)
    plt.plot(mean_k3d_real, mean_Pk_real + stdev_Pk_real, color="b", linewidth=2.5, alpha=0.4, linestyle="--")
    plt.plot(mean_k3d_real, mean_Pk_real - stdev_Pk_real, color="b", linewidth=2.5, alpha=0.4, linestyle="--")
    
    plt.title("Power Spectrum Comparison")
    plt.xlabel('log(Pk.k3D)')
    plt.ylabel('log(Pk)')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.09), fancybox=True, shadow=True, ncol=3)
    
    plt.show()

